In [1]:
%run startup.py
bf = Session(host="localhost")

In [2]:
# Assign a friendly name to your network and snapshot
BASE_NETWORK_NAME = "T2"
BASE_SNAPSHOT_NAME = "T2_whole_topo"

BASE_SNAPSHOT_PATH = "./networks/t2"

# Now create the network and initialize the snapshot
bf.set_network(BASE_NETWORK_NAME)
bf.init_snapshot(BASE_SNAPSHOT_PATH, name=BASE_SNAPSHOT_NAME, overwrite=True)

'T2_whole_topo'

In [3]:
# Fork a new snapshot with backup links inactive
BLINKS_INACTIVE_SNAPSHOT_NAME = "two_b_links_inactive"

interfaces = [
    Interface(hostname="r_1", interface="Ethernet2"),
    Interface(hostname="r_v", interface="Ethernet2"),
    Interface(hostname="r_2", interface="Ethernet3"),
    Interface(hostname="r_w", interface="Ethernet2")
    ]
bf.fork_snapshot(BASE_SNAPSHOT_NAME, BLINKS_INACTIVE_SNAPSHOT_NAME, deactivate_interfaces=interfaces, overwrite=True)

'two_b_links_inactive'

In [4]:
# Do a traceroute from host1 to 1.0.2.2
tr_frame = bf.q.traceroute(startLocation="/source/", headers=HeaderConstraints(dstIps="r_w")).answer(BLINKS_INACTIVE_SNAPSHOT_NAME).frame()

# Display results using customizations to handle large string values

show(tr_frame)

,Flow,Traces,TraceCount
0,Start Location: source1Src IP: 10.0.4.20Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"DENIED_OUT1. node: source1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.0.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.0.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet1)3. node: r_u RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.1.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.1.2)]) TRANSMITTED(Ethernet1)4. node: r_2 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.2.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.2.2)]) DENIED(ACL2 (EGRESS_FILTER))",1
1,Start Location: source2Src IP: 10.0.4.25Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: source2 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.0.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.0.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet1)3. node: r_u RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.1.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.1.2)]) TRANSMITTED(Ethernet1)4. node: r_2 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.2.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.2.2)]) PERMITTED(ACL2 (EGRESS_FILTER)) TRANSMITTED(Ethernet1)5. node: r_v RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1, Routes: [connected (Network: 10.0.3.0/24, Next Hop: interface Ethernet1)]) TRANSMITTED(Ethernet1)6. node: r_w RECEIVED(Ethernet0) ACCEPTED(Ethernet0)",1


In [5]:
result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = '/source/'), headers=HeaderConstraints(dstIps='r_w'), actions='SUCCESS').answer().frame()

result.Flow

0    start=source2 [10.0.4.25->10.0.3.2 ICMP (type=8, code=0)]
Name: Flow, dtype: object

In [6]:
# link(1,u) fail
# Fork a new snapshot with backup links inactive
LINK_1u_INACTIVE_SNAPSHOT_NAME = "link_1u_inactive"

interfaces = [
    Interface(hostname="r_1", interface="Ethernet1"),
    Interface(hostname="r_u", interface="Ethernet0"),
    Interface(hostname="r_2", interface="Ethernet3"),
    Interface(hostname="r_w", interface="Ethernet2")
    ]
bf.fork_snapshot(BASE_SNAPSHOT_NAME, LINK_1u_INACTIVE_SNAPSHOT_NAME, deactivate_interfaces=interfaces, overwrite=True)

'link_1u_inactive'

In [7]:
# Do a traceroute from host1 to 1.0.2.2
tr_frame = bf.q.traceroute(startLocation="/source/", headers=HeaderConstraints(dstIps="r_w")).answer(LINK_1u_INACTIVE_SNAPSHOT_NAME).frame()

# Display results using customizations to handle large string values

show(tr_frame)

,Flow,Traces,TraceCount
0,Start Location: source1Src IP: 10.0.4.20Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: source1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet2 with resolved next-hop IP: 10.0.6.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet2 ip 10.0.6.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet2)3. node: r_v RECEIVED(Ethernet2) FORWARDED(Forwarded out interface: Ethernet1, Routes: [connected (Network: 10.0.3.0/24, Next Hop: interface Ethernet1)]) TRANSMITTED(Ethernet1)4. node: r_w RECEIVED(Ethernet0) ACCEPTED(Ethernet0)",1
1,Start Location: source2Src IP: 10.0.4.25Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: source2 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet2 with resolved next-hop IP: 10.0.6.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet2 ip 10.0.6.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet2)3. node: r_v RECEIVED(Ethernet2) FORWARDED(Forwarded out interface: Ethernet1, Routes: [connected (Network: 10.0.3.0/24, Next Hop: interface Ethernet1)]) TRANSMITTED(Ethernet1)4. node: r_w RECEIVED(Ethernet0) ACCEPTED(Ethernet0)",1


In [9]:
result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = '/source/'), headers=HeaderConstraints(dstIps='r_w'), actions='SUCCESS').answer(LINK_1u_INACTIVE_SNAPSHOT_NAME).frame()

result.Flow

0    start=source1 [10.0.4.20->10.0.3.2 ICMP (type=8, code=0)]
1    start=source2 [10.0.4.25->10.0.3.2 ICMP (type=8, code=0)]
Name: Flow, dtype: object

In [10]:
# link(2,v) fail
# Fork a new snapshot with backup links inactive
LINK_2v_INACTIVE_SNAPSHOT_NAME = "link_2v_inactive"

interfaces = [
    Interface(hostname="r_1", interface="Ethernet2"),
    Interface(hostname="r_v", interface="Ethernet2"),
    Interface(hostname="r_2", interface="Ethernet1"),
    Interface(hostname="r_v", interface="Ethernet0")
    ]
bf.fork_snapshot(BASE_SNAPSHOT_NAME, LINK_2v_INACTIVE_SNAPSHOT_NAME, deactivate_interfaces=interfaces, overwrite=True)

'link_2v_inactive'

In [11]:
# Do a traceroute from host1 to 1.0.2.2
tr_frame = bf.q.traceroute(startLocation="/source/", headers=HeaderConstraints(dstIps="r_w")).answer(LINK_2v_INACTIVE_SNAPSHOT_NAME).frame()

# Display results using customizations to handle large string values

show(tr_frame)

,Flow,Traces,TraceCount
0,Start Location: source1Src IP: 10.0.4.20Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"DENIED_OUT1. node: source1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.0.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.0.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet1)3. node: r_u RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.1.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.1.2)]) TRANSMITTED(Ethernet1)4. node: r_2 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet3 with resolved next-hop IP: 10.0.7.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet3 ip 10.0.7.2)]) DENIED(ACL2 (EGRESS_FILTER))",1
1,Start Location: source2Src IP: 10.0.4.25Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: source2 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.0.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.0.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet1)3. node: r_u RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet1 with resolved next-hop IP: 10.0.1.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet1 ip 10.0.1.2)]) TRANSMITTED(Ethernet1)4. node: r_2 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet3 with resolved next-hop IP: 10.0.7.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet3 ip 10.0.7.2)]) PERMITTED(ACL2 (EGRESS_FILTER)) TRANSMITTED(Ethernet3)5. node: r_w RECEIVED(Ethernet2) ACCEPTED(Ethernet0)",1


In [12]:
result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = '/source/'), headers=HeaderConstraints(dstIps='r_w'), actions='SUCCESS').answer(LINK_2v_INACTIVE_SNAPSHOT_NAME).frame()

result.Flow

0    start=source2 [10.0.4.25->10.0.3.2 ICMP (type=8, code=0)]
Name: Flow, dtype: object

In [16]:
# 2 links fail
# Fork a new snapshot with backup links inactive
LINK_2link_INACTIVE_SNAPSHOT_NAME = "link_2link_inactive"

interfaces = [
    Interface(hostname="r_1", interface="Ethernet1"),
    Interface(hostname="r_u", interface="Ethernet0"),
    Interface(hostname="r_2", interface="Ethernet1"),
    Interface(hostname="r_v", interface="Ethernet0")
    ]
bf.fork_snapshot(BASE_SNAPSHOT_NAME, LINK_2link_INACTIVE_SNAPSHOT_NAME, deactivate_interfaces=interfaces, overwrite=True)

'link_2link_inactive'

In [17]:
# Do a traceroute from host1 to 1.0.2.2
tr_frame = bf.q.traceroute(startLocation="/source/", headers=HeaderConstraints(dstIps="r_w")).answer(LINK_2link_INACTIVE_SNAPSHOT_NAME).frame()

# Display results using customizations to handle large string values

show(tr_frame)

,Flow,Traces,TraceCount
0,Start Location: source1Src IP: 10.0.4.20Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: source1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet2 with resolved next-hop IP: 10.0.6.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet2 ip 10.0.6.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet2)3. node: r_v RECEIVED(Ethernet2) FORWARDED(Forwarded out interface: Ethernet1, Routes: [connected (Network: 10.0.3.0/24, Next Hop: interface Ethernet1)]) TRANSMITTED(Ethernet1)4. node: r_w RECEIVED(Ethernet0) ACCEPTED(Ethernet0)",1
1,Start Location: source2Src IP: 10.0.4.25Src Port: 49152Dst IP: 10.0.3.2Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: source2 ORIGINATED(default) FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.0.4.2, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.0.4.2)]) TRANSMITTED(eth0)2. node: r_1 RECEIVED(Ethernet0) FORWARDED(Forwarded out interface: Ethernet2 with resolved next-hop IP: 10.0.6.2, Routes: [ospf (Network: 10.0.3.0/24, Next Hop: interface Ethernet2 ip 10.0.6.2)]) PERMITTED(ACL1 (EGRESS_FILTER)) TRANSMITTED(Ethernet2)3. node: r_v RECEIVED(Ethernet2) FORWARDED(Forwarded out interface: Ethernet1, Routes: [connected (Network: 10.0.3.0/24, Next Hop: interface Ethernet1)]) TRANSMITTED(Ethernet1)4. node: r_w RECEIVED(Ethernet0) ACCEPTED(Ethernet0)",1


In [18]:
result = bf.q.reachability(pathConstraints=PathConstraints(startLocation = '/source/'), headers=HeaderConstraints(dstIps='r_w'), actions='SUCCESS').answer(LINK_2link_INACTIVE_SNAPSHOT_NAME).frame()

result.Flow

0    start=source1 [10.0.4.20->10.0.3.2 ICMP (type=8, code=0)]
1    start=source2 [10.0.4.25->10.0.3.2 ICMP (type=8, code=0)]
Name: Flow, dtype: object